
##  세종대학교 인공지능 수업 선형 회의 과제 입니다.

## 선형회귀 - 학습용 코드

### 과제의 목표
- 배추 생산량은 온도와 강우량에 영향을 받고, 그 생산량에 의해 배추 가격이 형성됩니다.

- 내일의 온도와 강우량으로 배추가격을 추측해봅시다.


### 선형 회귀 학습에 사용할 데이터 보기
- [배추 가격 데이터 보기](https://drive.google.com/file/d/1-6SShVMc3WwXFcrV7TGtV9ekZPB9Pf3Z/view?usp=sharing)


### 주의

- 개인 구글 드라이브 저장소 사용을 위한 설정

## [1] 개인 구글 드라이브의 저장소를 Colab 에서 사용하기 위한 설정

### 관련 코드
```
from google.colab import drive
drive.mount('/content/gdrive')

```

###  본 설정이 필요한 이유?
- Colab의 세션 유지 시간은 약 6시간이다.
- 세션이 새롭게 갱신될때마다 할당되는 컴퓨터가 달라짐
  - PC방 바탕화면에 파일 다운로드 후 재부팅하면 컴퓨터에 파일이 하나도 남아 있지 않는 상황과 도일
- 따라서 본인의 구글 드라이브를 마운트 해서 파일 저장, 불러오기가 이루어져야 함



In [3]:
from google.colab import drive
import os

if os.path.exists('/content/gdrive')==False:
  drive.mount('/content/gdrive')
  print('Google Drive is mounted\n')
else:
  print('Google Drive is already mounted\n')

Google Drive is already mounted



## [2] 학습데이터 다운로드 & 학습

### 다운로드 관련 
- 리눅스의 curl 이라는 명령어를 활용하여 다운로드

### 학습 관련
- 학습하기 위한 W, b  값을 찾기 위해, 최적화 반복횟수는 얼마든지 조절 가능


In [4]:
import os
import tensorflow as tf
import numpy as np
from pandas.io.parsers import read_csv

model = tf.global_variables_initializer()

if os.path.exists('/content/gdrive/My Drive/Colab Notebooks/price_data_yk.csv')==False:
  print('Train data downloading..')
  ! curl 'https://raw.githubusercontent.com/unizard/2019.Spring.AI/master/price_data_yk.csv' -o '/content/gdrive/My Drive/Colab Notebooks/price_data_yk.csv'
  print('Done..\n')
else:
  print('File already exists \n')
    
# 학습데이터 로딩
data = read_csv('/content/gdrive/My Drive/Colab Notebooks/price_data_yk.csv',sep=',')
xy = np.array(data,dtype=np.float32)


# 4개의 변인을 입력을 받습니다.
x_data = xy[:, 1:-1]

# 가격 값을 입력 받습니다.
y_data = xy[:, [-1]]

# 플레이스 홀더를 설정합니다.
X = tf.placeholder(tf.float32, shape=[None, 4])
Y = tf.placeholder(tf.float32, shape=[None, 1])

W = tf.Variable(tf.random_normal([4, 1]), name="weight")
b = tf.Variable(tf.random_normal([1]), name="bias")

# 가설을 설정합니다.
# 
hypothesis = tf.matmul(X,W) + b

# 비용 함수를 설정합니다.
cost = tf.reduce_mean(tf.square(hypothesis-Y))

# 최적화 함수를 설정합니다.
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.000005)
train = optimizer.minimize(cost)

# 세션을 생성합니다.
sess = tf.Session()

# 글로벌 변수를 초기화합니다.
sess.run(tf.global_variables_initializer())

# 학습을 수행합니다.
print('=========================================')
print(' 학습 시작')
print('=========================================')

for step in range(100001):
    cost_, hypo_, _ = sess.run([cost, hypothesis, train], feed_dict={X: x_data, Y: y_data})
    if step % 10000 == 0:
        print("#", step, " 손실 비용: ", cost_)
        print("- 배추 가격: ", hypo_[0])


print('=========================================')
print(' 학습 끝')
print('=========================================')


Train data downloading..
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 93044  100 93044    0     0   447k      0 --:--:-- --:--:-- --:--:--  447k
Done..

Instructions for updating:
Colocations handled automatically by placer.
 학습 시작
# 0  손실 비용:  12457230.0
- 배추 가격:  [-11.229731]
# 10000  손실 비용:  2298854.8
- 배추 가격:  [2269.5767]
# 20000  손실 비용:  2271106.8
- 배추 가격:  [2549.9558]
# 30000  손실 비용:  2265283.2
- 배추 가격:  [2577.4253]
# 40000  손실 비용:  2259743.2
- 배추 가격:  [2580.5674]
# 50000  손실 비용:  2254286.5
- 배추 가격:  [2581.2693]
# 60000  손실 비용:  2248915.2
- 배추 가격:  [2582.2112]
# 70000  손실 비용:  2243615.2
- 배추 가격:  [2583.0107]
# 80000  손실 비용:  2238396.2
- 배추 가격:  [2583.5032]
# 90000  손실 비용:  2233233.8
- 배추 가격:  [2583.8347]
# 100000  손실 비용:  2228164.5
- 배추 가격:  [2584.5195]
 학습 끝


## [3] 학습 모델 저장


### 구글 드라이브에 모델 저장
- 마운트한 경로 잘 설정하여 저장

In [5]:
# 오늘의 날짜를 파일 이름으로 활용할 예정이었으나, 여러분의 혼란을 줄이고자 주석처리함
# 오늘의 날짜를 사용하기 위한 라이브러리 import
import datetime


# 학습 모델 저장을 위한 그래프 설정
saver = tf.train.Saver()

# 학습 모델 이름 설정
#strTLog = datetime.date.today().strftime("%Y%m%d")  
path = '/content/gdrive/My Drive/Colab Notebooks/saved.cpkt'

# 학습 모델 저장
save_path = saver.save(sess,path)
print('학습된 모델을 저장했습니다.')


학습된 모델을 저장했습니다.
